# oneTimePad.py

Here is the source code.

```
#!/usr/bin/env python
# coding=utf-8

from os import urandom

def process(m, k):
    tmp = m ^ k
    res = 0
    for i in bin(tmp)[2:]:
        res = res << 1;
        if (int(i)):
            res = res ^ tmp
        if (res >> 256):
            res = res ^ P
    return res

def keygen(seed):
    key = str2num(urandom(32))
    while True:
        yield key
        key = process(key, seed)

def str2num(s):
    return int(s.encode('hex'), 16)

P = 0x10000000000000000000000000000000000000000000000000000000000000425

true_secret = open('flag.txt').read()[:32]
assert len(true_secret) == 32
print('flag{%s}' % true_secret)
fake_secret1 = "I_am_not_a_secret_so_you_know_me"
fake_secret2 = "feeddeadbeefcafefeeddeadbeefcafe"
secret = str2num(urandom(32))

generator = keygen(secret)
ctxt1 = hex(str2num(true_secret) ^ generator.next())[2:-1]
ctxt2 = hex(str2num(fake_secret1) ^ generator.next())[2:-1]
ctxt3 = hex(str2num(fake_secret2) ^ generator.next())[2:-1]
f = open('ciphertext', 'w')
f.write(ctxt1+'\n')
f.write(ctxt2+'\n')
f.write(ctxt3+'\n')
f.close()
```

As you see, the flag is `true_sectet`. But the `flag.txt` file lost and we must use `ciphertext` to reverse inference the flag.

At first, let's see `str2num`. It encodes a string in form of `hex` and transfer to an integer.

```
str2num(s) = int(s.encode('hex'), 16)
```

So we can write the reverse function:

```
num2str(n) = str(hex(n)).decode('hex')
```

But the result of `hex()` has the format `r/^0x[0-9]+L$/`. So we should strip the head and tail.

In [19]:
def num2str(n):
    return str(hex(n))[2:-1].decode('hex')
def str2num(s):
    return int(s.encode('hex'), 16)

Now we concerned about the process of ctxt:

```
ctxt = hex(str2num(secret) ^ generator)[2:-1]
```
Notice the property of `XOR`, we can reverse the secret:
```
secret = int(ctxt, 16) ^ generator 
```

In [20]:
def crypto(ctxt, secret):
    return int(ctxt, 16) ^ str2num(secret)

Now we can get the `ctxt3` and `ctxt2` according to `fake_secret`, and we will get the generator of `secret` by these two generator.

In [21]:
fake_secret1 = "I_am_not_a_secret_so_you_know_me"
fake_secret2 = "feeddeadbeefcafefeeddeadbeefcafe"
ctxt2 = "630eb4dce274d29a16f86940f2f35253477665949170ed9e8c9e828794b5543c"
ctxt3 = "e913db07cbe4f433c7cdeaac549757d23651ebdccf69d7fbdfd5dc2829334d1b"
generator2 = crypto(ctxt2, fake_secret1)
generator3 = crypto(ctxt3, fake_secret2)

LookupError: 'hex' is not a text encoding; use codecs.encode() to handle arbitrary codecs

Finally, we just need to concentrate on the `process()`!  
We can get the `seed` by two generators, and then get the real generator by the `seed` and the `generator2`!

In [ ]:
P = 0x10000000000000000000000000000000000000000000000000000000000000425L
def getSeed(g1, g2):
    i = 0
    while g2 != 0:
        if ((g2 ^ P) >> 256):
            g2 = g2 ^ 256
    # TODO